<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_AWS_APRIL2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env --quiet

!pip install sagemaker boto3 --quiet

#%pip install langchain==0.0.309 --quiet --root-user-action=ignore
%pip install langchain --quiet

import colab_env
import os

In [ ]:
aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
region=os.getenv("AWS_DEFAULT_REGION")
output=os.getenv("AWS_DEFAULT_OUTPUT")

#print(aws_access_key_id)
#print()
#print(f"aws_access_key_id: '{aws_access_key_id}'")
#print(f"aws_secret_access_key: '{aws_secret_access_key}'")

#print(f"region: '{region}'")
#print()

llama-2
https://aws.amazon.com/blogs/machine-learning/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/

claude-3
https://aws.amazon.com/blogs/aws/anthropics-claude-3-sonnet-foundation-model-is-now-available-in-amazon-bedrock/

mistral 8x7b https://aws.amazon.com/blogs/machine-learning/mixtral-8x7b-is-now-available-in-amazon-sagemaker-jumpstart/


In [ ]:
import colab_env
import boto3
import os
import sagemaker
from sagemaker.jumpstart.model import JumpStartModel

iam_client = boto3.client("iam")

role = iam_client.get_role(
    RoleName=os.getenv("ROLENAME")
)

ROLE_ARN = role['Role']['Arn']


llm_model_id = 'huggingface-llm-mistral-7b'
llm_model_version = '2.1.0'


llm_model = JumpStartModel(model_id=llm_model_id, model_version=llm_model_version, role=ROLE_ARN, region='us-east-1')

llm_predictor = llm_model.deploy(accept_eula=True)


In [ ]:
#this is the model endpoint NAME, not the ARN
llm_model_endpoint_name = llm_predictor.endpoint_name
llm_model_endpoint_name

'hf-llm-mistral-7b-2025-04-07-07-32-25-889'

In [ ]:
#### MISTRAL ###### CASE#1
import json
#query = "who is the best French Poet?"
query = "Write a program to compute factorial in python:"


# Create a boto3 client for SageMaker runtime
sm_client = boto3.client('runtime.sagemaker')

# Prepare the input for the model
#input_data = {"inputs": query}

### WITH PARAMETRS
n=5
MNT=512*n
model_kwargs={"max_new_tokens": MNT, "temperature": 0.9}
input_data = ({"inputs": query, "parameters" : {**model_kwargs}})

response = sm_client.invoke_endpoint(EndpointName=llm_model_endpoint_name, Body=json.dumps(input_data), ContentType="application/json")

# Decode the response from the model
response_body = json.loads(response['Body'].read().decode('utf-8'))
#print(response_body)

print(f'Query:', query)
print()
print(f'Response:', response_body[0]['generated_text'])

Query: Write a program to compute factorial in python:

Response: 

Factorial of a given number is the product of that number and the numbers stemming from it up to 1. is a number!n ⁢ n − 1 ⁢ n − 2 ⁢ . ⁢ . ⁢ . ⁢ 1
where n is a non-negative integer.

Note: !n = 1 when n = 0.

You are required to design and implement the above algorithm in python.

#include <stdio.h>
int factorial(int number)
{
int ans = 1;
for (int i = 2; i <= number; i++) {
ans *= i;
}
return ans;
}
int main()
{
printf("%d\n", factorial(6));
return 0;
}

int factorial(int n)
{
if (n==0) return 1;
return n * factorial(n-1);
}
int main()
{
printf("%d\n", factorial(6));
return 0;
}


In [ ]:
#### MISTRAL ###### CASE#2

#query = "who is the best French Poet?"
query = "I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering."


# Create a boto3 client for SageMaker runtime
sm_client = boto3.client('runtime.sagemaker')

# Prepare the input for the model
#input_data = {"inputs": query}

### WITH PARAMETRS
n=5
MNT=512*n
model_kwargs={"max_new_tokens": MNT, "temperature": 0.9}
input_data = ({"inputs": query, "parameters" : {**model_kwargs}})

response = sm_client.invoke_endpoint(EndpointName=llm_model_endpoint_name, Body=json.dumps(input_data), ContentType="application/json")

# Decode the response from the model
response_body = json.loads(response['Body'].read().decode('utf-8'))
#print(response_body)

print(f'Query:', query)
print()
print(f'Response:', response_body[0]['generated_text'])

Query: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.

Response: 

Solution:-

Number of kids 6

Price of ice cream $1.25 each

Price = $1.25 * 6 = $7.50

Money paid =10

So, the amount of money remaining = $10-$7.50 = $2.50

So, the amount of money received is $2.50


# CLEAN UP

In [ ]:
#  Frank Morales created this cell on December 14, 2023; it fully allows automatically the deletion of endpoints, models, and endpoint configurations.

import colab_env
import os

aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region=os.getenv("AWS_DEFAULT_REGION")
aws_output=os.getenv("AWS_DEFAULT_OUTPUT")

import boto3

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

def cleanup_sagemaker_resources(resource_name,resourceid):

    if resourceid==0:
       response=sagemaker_client.list_endpoints()
    elif resourceid==1:
         response=sagemaker_client.list_models()
    elif resourceid==2:
         response=sagemaker_client.list_endpoint_configs()

    print(resource_name)

    number_of_endpoints=len(response['%s'%resource_name])
    for i in range(number_of_endpoints):
        resource_nametmp='%s'%resource_name[0:len(resource_name)-1]
        print('%sName'%resource_nametmp)
        print(response['%s'%resource_name][i]['%sName'%resource_nametmp])

        if resourceid==0:
           endpoint_name=response['%s'%resource_name][i]['%sName'%resource_nametmp]
           sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        elif resourceid==1:
           sagemaker_client.delete_model(ModelName=response['Models'][i]['ModelName'])
        elif resourceid==2:
           sagemaker_client.delete_endpoint_config(EndpointConfigName=response['EndpointConfigs'][i]['EndpointConfigName'])

    print("\n==================================\n")


cleanup_sagemaker_resources('Endpoints',0)
cleanup_sagemaker_resources('Models',1)
cleanup_sagemaker_resources('EndpointConfigs',2)

Endpoints
EndpointName
hf-llm-mistral-7b-2025-04-07-07-32-25-889


Models
ModelName
hf-llm-mistral-7b-2025-04-07-07-32-25-887


EndpointConfigs
EndpointConfigName
hf-llm-mistral-7b-2025-04-07-07-32-25-889


